In [1]:
import json
from collections import Counter

# 파일 경로 설정
clusters_by_id_file = "clusters_by_id.json"  # PUUID가 클러스터로 분류된 파일
most_common_synergies_file = "most_common_synergies.json"  # 13시즌 시너지 데이터
output_file = "cluster_synergies_output.json"  # 결과를 저장할 파일
filtered_output_file = "filtered_cluster_synergies_output.json"  # 전처리된 결과를 저장할 파일

# JSON 데이터 읽기 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 데이터 쓰기 함수
def save_json(data, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# 클러스터별 시너지 계산 함수
def process_clusters(clusters_data, synergies_data):
    cluster_results = {}

    for cluster_id, puuids in clusters_data.items():
        synergies_in_cluster = []

        for puuid in puuids:
            for synergy_record in synergies_data:
                if synergy_record.get("puuid") == puuid:
                    synergy_names = synergy_record.get("synergy_name")
                    if isinstance(synergy_names, list):
                        synergies_in_cluster.extend(synergy_names)
                    else:
                        synergies_in_cluster.append(synergy_names)

        # 전체 시너지 개수
        total_synergies = len(synergies_in_cluster)

        # 시너지 빈도 계산 및 백분율 계산
        synergy_counter = Counter(synergies_in_cluster)
        synergy_percentages = {
            synergy: {
                "count": count,
                "percentage": (count / total_synergies) * 100 if total_synergies > 0 else 0
            }
            for synergy, count in synergy_counter.items()
        }

        # 내림차순으로 정렬
        sorted_synergy_percentages = dict(sorted(synergy_percentages.items(), key=lambda item: item[1]["count"], reverse=True))

        # 가장 많이 사용된 시너지 계산
        most_common_synergy = synergy_counter.most_common(1)

        # 클러스터 결과 저장
        cluster_results[cluster_id] = {
            "synergy_statistics": sorted_synergy_percentages,
            "most_common_synergy": most_common_synergy[0][0] if most_common_synergy else None,
            "most_common_count": most_common_synergy[0][1] if most_common_synergy else 0
        }

    return cluster_results

# 클러스터별 상위 2개 시너지 필터링 함수
def filter_top_synergies(cluster_synergies):
    filtered_results = {}

    for cluster_id, data in cluster_synergies.items():
        sorted_synergies = list(data["synergy_statistics"].items())[:3]  # 상위 2개만 가져오기
        filtered_results[cluster_id] = {
            "top_synergies": {
                synergy: stats for synergy, stats in sorted_synergies
            }
        }

    return filtered_results

# 실행
if __name__ == "__main__":
    try:
        # JSON 데이터 로드
        clusters_data = load_json(clusters_by_id_file)
        synergies_data = load_json(most_common_synergies_file)

        # 데이터 처리
        cluster_synergies = process_clusters(clusters_data, synergies_data)

        # 전처리: 상위 2개 시너지 필터링
        filtered_synergies = filter_top_synergies(cluster_synergies)

        # 결과 저장
        save_json(cluster_synergies, output_file)
        save_json(filtered_synergies, filtered_output_file)
        print(f"전체 결과가 {output_file} 파일에 저장되었습니다.")
        print(f"전처리된 결과가 {filtered_output_file} 파일에 저장되었습니다.")

    except Exception as e:
        print(f"오류 발생: {e}")


전체 결과가 cluster_synergies_output.json 파일에 저장되었습니다.
전처리된 결과가 filtered_cluster_synergies_output.json 파일에 저장되었습니다.
